In [1]:
import sys
import os
from pathlib import Path
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
sys.path.append(os.path.join(str(Path(os.getcwd()).parent), 'NEAT_LNC_node'))
import torch
import NEAT_LNC_node as neat
import pprint
from utils import *
from configuration import *
from NEAT_args import *

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')
    
args = parser.parse_args([])
EA_args = EA_parser.parse_args([])

In [3]:
results = torch.zeros([13,10,3,4])

args.DEVICE = 'cpu'

dic = {'ptanh': 0,
       'sigmoid': 1,
       'hardsigmoid': 2,
       'prelu': 3}

for ds in range(13):
    args.DATASET = ds
    args = FormulateArgs(args)
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
    pprint.pprint(datainfo)
    
    for seed in range(10):
        args.SEED = seed
        for e, eps_train in enumerate([0.0, 0.05, 0.1]):
            args.e_train = eps_train

            modelname = f"NEAT_data_{datainfo['dataname']}_seed_{args.SEED:02d}_epsilon_{eps_train}.model"
            modelpath = f'./models/{modelname}'
            if os.path.exists(modelpath):
                model = torch.load(modelpath)

            for n in model.nodes:
                act_temp = model.nodes[n].Activation

                results[ds, seed, e, dic[act_temp]] += 1

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 79,
 'N_train': 373,
 'N_valid': 124,
 'dataname': 'balancescale'}
{'N_class': 2,
 'N_feature': 9,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'breastcancerwisc'}
{'N_class': 3,
 'N_feature': 21,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'cardiotocography3clases'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy1'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy2'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 31,
 'N_train': 88,
 'N_valid': 29,
 'dataname': 'iris'}
{'N_class': 2,
 'N_feature': 5,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'mammographic'}
{'N_class': 10,
 'N_feature': 16,
 'N_test': 79,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'pendigits'}
{'N_class'

In [4]:
re = results.sum(1)
re.shape

torch.Size([13, 3, 4])

In [5]:
N_neurons =  re.sum(2)
print(N_neurons.shape)
N_neurons

torch.Size([13, 3])


tensor([[ 42.,  45.,  93.],
        [ 53.,  34.,  60.],
        [ 56.,  26.,  35.],
        [ 82.,  44.,  49.],
        [ 58.,  37.,  70.],
        [ 79.,  94., 140.],
        [ 91., 113., 143.],
        [ 74.,  82.,  45.],
        [177., 233., 255.],
        [ 74.,  84., 115.],
        [ 39.,  50., 100.],
        [ 57.,  78.,  85.],
        [ 65.,  59.,  81.]])

In [6]:
re_norm = re / N_neurons[:,:,None]
re_norm.shape

torch.Size([13, 3, 4])

In [7]:
re_norm[0]

tensor([[0.6429, 0.1667, 0.1429, 0.0476],
        [0.4444, 0.1778, 0.2444, 0.1333],
        [0.3548, 0.1828, 0.2258, 0.2366]])

In [8]:
for ds in range(13):
    args.DATASET = ds
    args = FormulateArgs(args)
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')

    dataname = datainfo['dataname']
    np.savetxt(f'./evaluation/histogram_{ds:02d}_{dataname}.txt', re_norm[ds]*100, fmt='%.5f')

In [9]:
re_sum = re.sum(0)
re_sum

tensor([[520., 164., 131., 132.],
        [452., 171., 185., 171.],
        [477., 264., 266., 264.]])

In [10]:
N_neurons2 = re_sum.sum(1)
re_sum/ N_neurons2[:,None]

tensor([[0.5491, 0.1732, 0.1383, 0.1394],
        [0.4617, 0.1747, 0.1890, 0.1747],
        [0.3753, 0.2077, 0.2093, 0.2077]])